# Mission to Mars

### Resources:
* <a href='https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'>NASA Mars News Site</a>
* <a href='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'>JPL Featured Space Image</a>
* <a href='https://twitter.com/marswxreport?lang=en'>Mars Weather</a>
* <a href='https://space-facts.com/mars/'>Mars Facts</a>
* <a href='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'>Mars Hemispheres</a>

### Dependencies

In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
# URLs
url_nasa = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_weather = "https://twitter.com/marswxreport?lang=en"
url_facts = "https://space-facts.com/mars/"
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Chromedriver initiation

In [3]:
def init_browser():
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

### NASA Mars News Site

In [4]:
def scrape_nasa_mars_news(url):
    
    browser = init_browser()    
    browser.visit(url)
    html = browser.html
    soup = bs(html, "html.parser")
    
    news_title = soup.find(class_="content_title").get_text()
    news_text = soup.find(class_="article_teaser_body").get_text()
    
    news = {
            'news_title': news_title,
            'news_text': news_text
        }
    
    browser.quit()
    
    return news

### JPL Featured Space Image

In [5]:
def scrape_jpl_featured_space_image(url):
    
    browser = init_browser()
    browser.visit(url)

    browser.find_by_id('full_image').click()

    html = browser.html
    soup = bs(html, "html.parser")

    image = soup.find(class_='fancybox')['data-fancybox-href']

    featured_image_url = 'https://www.jpl.nasa.gov' + image
    featured_image_url
    
    featured_image = {
        'featured_image_url': featured_image_url
    }
    
    browser.quit()

    return featured_image

### Mars Weather

In [6]:
def scrape_mars_weather(url):
    
    browser = init_browser()
    browser.visit(url)
    html = browser.html
    soup = bs(html, "html.parser")

    weather = soup.find("p", class_="tweet-text").get_text()
    
    mars_weather = {
        'mars_weather_tweet_text': weather
    }
    
    browser.quit()
    
    return mars_weather

### Mars Facts

In [7]:
def scrape_mars_facts(url):
    
    browser = init_browser()
    browser.visit(url)
    html = browser.html
    soup = bs(html, "html.parser")

    table_html = soup.find("table")
    mars_facts_string = str(table_html)
    encoded_mars_facts = table_html.encode('utf8')

    mars_facts = {
        'string': mars_facts_string,
        'encoded': encoded_mars_facts
    }
    
    browser.quit()
    
    return mars_facts

In [12]:
url = url_facts
browser = init_browser()
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [13]:
table_html = soup.find("table")

In [14]:
mars_facts_string = str(table_html)
encoded_mars_facts = table_html.encode('utf8')

In [27]:
print(table_html)

<table class="tablepress tablepress-id-mars" id="tablepress-mars">
<tbody>
<tr class="row-1 odd">
<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>
</td>
</tr>
<tr class="row-2 even">
<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>
</td>
</tr>
<tr class="row-3 odd">
<td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>
</tr>
<tr class="row-4 even">
<td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>
</tr>
<tr class="row-5 odd">
<td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>
</tr>
<tr class="row-6 even">
<td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>
</td>
</tr>
<tr class="row-7 odd">
<td class

In [ ]:
mars_facts = {
    'string': mars_facts_string,
    'encoded': encoded_mars_facts
}

### Mars Hemispheres

In [8]:
def scrape_mars_hemispheres(url):

    browser = init_browser()
    browser.visit(url)
    html = browser.html
    soup = bs(html, "html.parser")

    links = []
    for a in soup.find_all('a', class_='itemLink product-item', href=True):
        if a['href'] not in links:
            links.append(a['href'])        

    hemispheres = {}
    
    for link in links:

        path = 'https://astrogeology.usgs.gov' + link
        browser.visit(path)
        soup.clear()
        html = browser.html
        soup = bs(html, "html.parser")

        title  = soup.find('h2').get_text()
        title_temp = title.replace(' Hemisphere Enhanced', '')
        title_clean = title_temp.replace(' ', '_')

        img_url = 'https://astrogeology.usgs.gov' + soup.find(class_='wide-image')['src']
        
        hemispheres[title_clean] = img_url
            
    browser.quit()
    
    return hemispheres


### Scrape function

In [9]:
def scrape_all():
    mars_data = {}
    
#     News
    news = scrape_nasa_mars_news(url_nasa)
    mars_data['news'] = news
    
#     JPL
    featured_image = scrape_jpl_featured_space_image(url_jpl)
    mars_data['jpl'] = featured_image
    
#     Weather
    mars_weather = scrape_mars_weather(url_weather)
    mars_data['weather'] = mars_weather
    
#     Facts
    mars_facts = scrape_mars_facts(url_facts)
    mars_data['facts'] = mars_facts
    
#     Hemispheres
    hemispheres = scrape_mars_hemispheres(url_hemispheres)
    mars_data['hemispheres'] = hemispheres
        
    return mars_data

In [10]:
mars_data = scrape_all()

In [11]:
print(mars_data)

{'news': {'news_title': 'NASA to Share Results of Effort to Recover Mars Rover', 'news_text': 'NASA will discuss the status of its Mars Exploration Rover Opportunity in a media briefing at 11 a.m. PST (2 p.m. EST) Wednesday, Feb. 13, from the Jet Propulsion Laboratory.'}, 'jpl': {'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17254_ip.jpg'}, 'weather': {'mars_weather_tweet_text': 'Sol 2315 (2019-02-09), high -17C/1F, low -71C/-95F, pressure at 8.13 hPa, daylight 06:47-18:52pic.twitter.com/xnGU2i98WB'}, 'facts': {'string': '<table class="tablepress tablepress-id-mars" id="tablepress-mars">\n<tbody>\n<tr class="row-1 odd">\n<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>\n</td>\n</tr>\n<tr class="row-2 even">\n<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>\n</td>\n</tr>\n<tr class="row-3 odd">\n<td class="column-1"><strong>Mass:</strong></td><td class="column-